### Install Dependencies

In [7]:
%pip install --q langchain-community
%pip install --q langchain
%pip install --q duckduckgo-search

# TODO: add the following packages to poetry
%pip install --q requests
%pip install --q beautifulsoup4

#################################
# Required for PaperSpace Gradient
%pip install --q typing-inspect==0.8.0 typing_extensions==4.5.0
%pip install --q pydantic==1.10.8

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Setup Ollama

In [16]:
# %> curl -fsSL https://ollama.com/install.sh | sh
# %> ollama serve
# %> ollama pull nomic-embed-text
# %> ollama pull mixtral:instruct     (gemma:7b-instruct | mistral:instruct)

!ollama list

NAME                   	ID          	SIZE  	MODIFIED       
mixtral:instruct       	7708c059a8bb	26 GB 	17 minutes ago	
nomic-embed-text:latest	0a109f422b47	274 MB	27 minutes ago	


### Accelerator Info

In [17]:
!nvidia-smi

Tue Feb 27 14:06:00 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:00:05.0 Off |                  Off |
| 30%   41C    P8    31W / 300W |      1MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Prompt

In [35]:
template = """{text}
-----------
Using the above text, answer in short the following question:
> {question}
-----------
if the question cannot be answered using the text, imply summarize the text. Include all
factual information, numbers, stats etc if available.
"""

In [36]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(template)

### Ingest Documents

In [14]:
import requests
from bs4 import BeautifulSoup

def scrape_url(url: str):
    try:
        # set a get request to webpage
        response = requests.get(url)

        # if the request response was successful
        if (response.status_code == 200):
            # parse the content of the request
            soup = BeautifulSoup(response.text, "html.parser")
            text_content = soup.get_text(separator=" ", strip=True)
            return text_content
        else:
            return f"Failed to retrieve page: {response.status_code}"
    except Exception as e:
        print(e)    

    return ""

In [15]:
url = "https://blog.langchain.dev/announcing-langsmith/"

content = scrape_url(url)[:10000]

### Setup Local LLM & Embedding Models

In [18]:
LLM_MODEL = "mixtral:instruct"  # ("gemma:7b-instruct" | "mistral:instruct")
EMBEDDING_MODEL = "nomic-embed-text"
TEMPERATURE = 0.9
ENABLE_TRACING = False
DOCUMENT_CHUNK_SIZE = 7500  # Gemma --> DOCUMENT_CHUNK_SIZE=5000
CHUNK_OVERLAP = 100

#### Load & Test Local LLM Model

#### Testing the Ollama model. No need to import separate llm model.

In [30]:
from langchain.llms import Ollama

_test_llm_model_ = Ollama(
    model=LLM_MODEL,
    temperature=TEMPERATURE,
)

_test_llm_model_("Who are you?")

' I am a large language model trained by Mistral AI. I was designed to answer questions and assist with tasks in a conversational manner, such as giving explanations or providing suggestions.'

### Setup Chat Chain

In [37]:
from langchain_community.chat_models import ChatOllama
from langchain.schema.output_parser import StrOutputParser

chain = prompt | ChatOllama(model=LLM_MODEL) | StrOutputParser()
chat_response = chain.invoke(
    {
        "text": content,
        "question": "What is LangSmith?"
    }
)
chat_response


' LangSmith is a unified platform for developing, debugging, testing, evaluating, and monitoring large language model (LLM) applications. It provides a framework for creating chains of LLMs and prompts, as well as tools for logging, tracing, and visualizing the inputs and outputs of these chains. LangSmith also includes features for creating datasets from traces or manual curation, running chains and prompts over those data sets, and evaluating their performance using heuristics or LLMs. It integrates seamlessly with open source evaluation modules and can be used to monitor system-level performance, track model/chain performance, debug issues, and understand user interactions. LangSmith is designed to help developers move from prototype to production more efficiently and effectively, and to provide a single hub for managing LLM applications.'